# Exercise 1

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Import CIFAR-10

In [14]:
transform = transforms.Compose([transforms.Resize(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root="../CIFAR10/",
                                        train=True,
                                        download=True,
                                        transform=transform)
train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                          shuffle=True)
validationloader = torch.utils.data.DataLoader(validationset,
                                          batch_size=1,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='../CIFAR10/',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=1,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


### CNN

In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True);

num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs,10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.0001)

writer = SummaryWriter()

Using cache found in /Users/simon/.cache/torch/hub/pytorch_vision_v0.10.0


### Training

In [19]:
for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(trainloader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(validationloader, 0):
        inputs, labels = data
        labels = labels

        outputs = model(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [10000/10000] - Loss: 2.07856297492980968, accuracy:  0.7728


NameError: name 'best_accuracy' is not defined

### Test

In [20]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    inputs, labels = data

    outputs = model(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.7667
